In [8]:
import csv
import pandas as pd

with open('Sentiment140.csv', mode='r') as file:
    tweet_data = csv.reader(file)
    
    labels = ['Polarity', 'Id', 'Date', 'Query', 'User', 'Text']
    df = pd.DataFrame(tweet_data, columns=labels)

df = df.drop(columns=['Id', 'Date', 'Query', 'User'], axis=1)

,Polarity,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...
